In [ ]:
import pandas as pd
import numpy as np
import time
import re
import pickle
from functools import reduce

In [ ]:
raw_training_data = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_logs.csv')
raw_training_scores = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv')

In [ ]:
raw_training_data.head(5)

In [ ]:
raw_training_data = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/test_logs.csv')

In [ ]:
raw_training_scores = pd.read_csv('/kaggle/input/linking-writing-processes-to-writing-quality/train_scores.csv')

Essay Recreator (credit given in comment at top)

In [ ]:
#####CODE FROM: https://www.kaggle.com/code/kawaiicoderuwu/essay-contructor

def getEssays(df):
    # Copy required columns
    textInputDf = df[['id', 'activity', 'cursor_position', 'text_change']].copy()
    
    # Get rid of text inputs that make no change
    # Note: Shift was unpreditcable so ignored
    textInputDf = textInputDf[textInputDf.activity != 'Nonproduction']

    # Get how much each Id there is
    valCountsArr = textInputDf['id'].value_counts(sort=False).values

    # Holds the final index of the previous Id
    lastIndex = 0

    # Holds all the essays
    essaySeries = pd.Series()

    # Fills essay series with essays
    for index, valCount in enumerate(valCountsArr):

        # Indexes down_time at current Id
        currTextInput = textInputDf[['activity', 'cursor_position', 'text_change']].iloc[lastIndex : lastIndex + valCount]

        # Update the last index
        lastIndex += valCount

        # Where the essay content will be stored
        essayText = ""

        
        # Produces the essay
        for Input in currTextInput.values:
            
            # Input[0] = activity
            # Input[2] = cursor_position
            # Input[3] = text_change
            
            # If activity = Replace
            if Input[0] == 'Replace':
                # splits text_change at ' => '
                replaceTxt = Input[2].split(' => ')
                
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                continue

                
            # If activity = Paste    
            if Input[0] == 'Paste':
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                continue

                
            # If activity = Remove/Cut
            if Input[0] == 'Remove/Cut':
                # DONT TOUCH
                essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                continue

                
            # If activity = Move...
            if "M" in Input[0]:
                # Gets rid of the "Move from to" text
                croppedTxt = Input[0][10:]
                
                # Splits cropped text by ' To '
                splitTxt = croppedTxt.split(' To ')
                
                # Splits split text again by ', ' for each item
                valueArr = [item.split(', ') for item in splitTxt]
                
                # Move from [2, 4] To [5, 7] = (2, 4, 5, 7)
                moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))

                # Skip if someone manages to activiate this by moving to same place
                if moveData[0] != moveData[2]:
                    # Check if they move text forward in essay (they are different)
                    if moveData[0] < moveData[2]:
                        # DONT TOUCH
                        essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                    else:
                        # DONT TOUCH
                        essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                continue
                
                
            # If just input
            # DONT TOUCH
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]

            
        # Sets essay at index  
        essaySeries[index] = essayText
     
    
    # Sets essay series index to the ids
    essaySeries.index =  textInputDf['id'].unique()
    
    
    # Returns the essay series
    return essaySeries

Fuctions to Create Features

In [ ]:


def scale_within_group(group):
        min_value = group['cursor_position'].min()
        max_value = group['cursor_position'].max()
        group['cursor_position'] = group['cursor_position'] - min_value
        return group
    
def segment_df_creator(df, num_segments):

        segment_dict = {}
        last_action_time = 1860000
        interval_len = int(round(last_action_time/num_segments, 0))


        for i in range(num_segments):
            if i == 0: 
                segment_dict[str(i)] = df[df['up_time'] < interval_len].reset_index()


            elif i == max(range(num_segments)):
                segment_dict[str(i)] = df[df['up_time'] >interval_len*max(range(num_segments))].reset_index()

            else: 
                segment_dict[str(i)] = df[(df['up_time']>interval_len*(i-1)) & (df['up_time']<interval_len*(i)) ] .reset_index()

        return segment_dict
    
def column_list(df, addendum):
    cols = list(df.columns)
    new_cols = []
    for col in cols:
        if col != 'id':
            new_cols.append(f'{col}_{addendum}')
        else: 
            new_cols.append(col)
    return new_cols
        

def get_revision_ratio(df):
    # Filter rows where 'activity' is either "Remove/Cut" or "Input"
    filtered_df = df[df['activity'].isin(["Remove/Cut", "Input"])]

    # Calculate revise count and activity count for respective conditions
    revise_count = filtered_df.loc[filtered_df['activity'] == "Remove/Cut", 'text_change'].str.len().sum()
    activity_count = filtered_df.loc[filtered_df['activity'] == "Input", 'text_change'].str.len().sum()

    # Calculate revision ratio
    if revise_count + activity_count == 0:
        return 0
    else: 
        return revise_count / (revise_count + activity_count)


def punctuation_ratios(df):
    
    punct_dict = {}
    try:
        comma_count = len(df[(df['activity'] == "Input") &(df['text_change'] == ",")])
        comma_minus = len(df[(df['activity'] == "Remove/Cut") &(df['text_change'] == ",")])
        punct_dict['comma_total'] = comma_count-comma_minus
    except: punct_dict['comma_total'] = 0
    try: 
        period_count = len(df[(df['activity'] == "Input") &(df['text_change'] == ".")])
        period_minus = len(df[(df['activity'] == "Remove/Cut") &(df['text_change'] == ".")])
        punct_dict['period_total'] = period_count-period_minus
        
    except: punct_dict['period_total'] = 0
    try: 
        dash_count = len(df[(df['activity'] == "Input") &(df['text_change'] == "-")])
        dash_minus = len(df[(df['activity'] == "Remove/Cut") &(df['text_change'] == "-")])
        punct_dict['dash_total'] = dash_count-dash_minus
    except: punct_dict['dash_total'] = 0
    try:
        qm_count = len(df[(df['activity'] == "Input") &(df['text_change'] == "?")])
        qm_minus = len(df[(df['activity'] == "Remove/Cut") &(df['text_change'] == "?")])
        punct_dict['qm_total'] =  qm_count-qm_minus
        
    except: punct_dict['qm_total'] = 0
    try:
        ep_count = len(df[(df['activity'] == "Input") &(df['text_change'] == "!")])
        ep_minus = len(df[(df['activity'] == "Remove/Cut") &(df['text_change'] == "!")])
        punct_dict['ep_total'] =  ep_count-ep_minus
    except: punct_dict['ep_total'] = 0
    try: 
        sc_count = len(df[(df['activity'] == "Input") &(df['text_change'] == ";")])
        sc_minus = len(df[(df['activity'] == "Remove/Cut") &(df['text_change'] == ";")])
        punct_dict['sc_total'] =  sc_count-sc_minus
    except: punct_dict['sc_total'] = 0
    try: 
        colon_count = len(df[(df['activity'] == "Input") &(df['text_change'] == ":")])
        colon_minus = len(df[(df['activity'] == "Remove/Cut") &(df['text_change'] == ":")])
        punct_dict['colon_total'] =  colon_count-colon_minus
    except: punct_dict['colon_total'] = 0
     
    try:
        ratio_dict = {}
        sumpunct = 0
        for key2 in punct_dict.keys():
                sumpunct += punct_dict[key2]
        for key in punct_dict.keys():
            ratio_dict[key.split('_')[0]] = punct_dict[key]/sumpunct

        return ratio_dict
    except: 
        for key in punct_dict.keys():
            ratio_dict[key.split('_')[0]] = 0
        return  ratio_dict
    
    
def sentence_length_stats(essay):
    try: 
        sentences = re.split(r'[.!?]', essay)
        # Remove empty segments if any
        sentences = [sentence for sentence in sentences if sentence]
        word_counts = []
        for sentence in sentences: 
            word_counts.append(sentence.count(' '))
        mean = np.mean(word_counts)
        sd = np.std(word_counts)
        return mean, sd
    except: 
        return 0,0

def word_len_stats(essay):
    try:
        words = re.findall(r'\w+|\S', essay)
        words = [word for word in words if word]

        # Count the number of 'q's in each word to represent the word length
        word_lengths = [word.count('q') for word in words if word.isalpha()]

        mean = sum(word_lengths) / len(word_lengths)

        # Calculate the standard deviation of word length
        sd = np.std(word_lengths)
    
        return mean, sd
    
    except: 
        return 0,0
    
def create_sum_stats(df):
    sum_df = df.groupby('id', as_index=False).agg(
    max_word_count=('word_count', 'max'), 
     min_word_count=('word_count', 'min'),
     median_word_count = ('word_count', 'median'),
     mean_word_count = ('word_count', 'mean'),
     std_word_count = ('word_count', 'std'), 
     skew_word_count = ('word_count', 'skew'),
     max_action_time=('action_time', 'max'), 
     min_action_time=('action_time', 'min'),
     median_action_time = ('action_time', 'median'),
     mean_action_time = ('action_time', 'mean'),
     std_action_time = ('action_time', 'std'), 
     skew_action_time = ('action_time', 'skew'),
     max_inter_action_time=('inter_action_time', 'max'), 
     min_inter_action_time=('inter_action_time', 'min'),
     median_inter_action_time = ('inter_action_time', 'median'),
     mean_inter_action_time = ('inter_action_time', 'mean'),
     std_inter_action_time = ('inter_action_time', 'std'), 
     skew_inter_action_time = ('inter_action_time', 'skew'))
    return sum_df.reset_index()

def essay_output_to_df(series):
    essay_summary_df = series.to_frame(name='essay')
    essay_summary_df['id'] = essay_summary_df.index
    essay_summary_df.reset_index(inplace = True)
    del(essay_summary_df['index'])
    essay_summary_df['mean_word_len'] = essay_summary_df['essay'].apply(lambda x: word_len_stats(x)[0])
    essay_summary_df['std_word_len'] = essay_summary_df['essay'].apply(lambda x: word_len_stats(x)[1])
    essay_summary_df['mean_sentence_len'] = essay_summary_df['essay'].apply(lambda x: sentence_length_stats(x)[0])
    essay_summary_df['std_sentence_len'] = essay_summary_df['essay'].apply(lambda x: sentence_length_stats(x)[0])
    del(essay_summary_df['essay'])
    return essay_summary_df

def calc_output_to_df(df):
    punct_results = df.groupby('id').apply(punctuation_ratios).reset_index()
    punct_summary_df = pd.DataFrame(punct_results.iloc[:, 1].tolist())
    punct_summary_df['id'] = punct_results['id']

    revision_results = df.groupby('id').apply(get_revision_ratio).reset_index()
    revision_results.columns = ['id', 'revision_ratio']
    
    total_rows_per_id = df.groupby('id').size().reset_index()
    total_rows_per_id.columns = ['id', 'total_actions']
    
    one_min_pauses = df[df['inter_action_time'] > 60000].groupby('id')['inter_action_time'].count().reset_index()
    one_min_pauses.columns = ['id', 'one_min_pause_count']
    one_min = pd.merge(one_min_pauses,total_rows_per_id, how = 'left', on = 'id')
    one_min['one_min_pause_to_output_ratio'] = 0  # Initialize ratio column with zeros
    one_min.loc[one_min['total_actions'] != 0, 'one_min_pause_to_output_ratio'] = one_min['one_min_pause_count'] / one_min['total_actions']
    del(one_min['total_actions'])

    ten_sec_pauses = df[df['inter_action_time'] > 10000].groupby('id')['inter_action_time'].count().reset_index()
    ten_sec_pauses.columns = ['id', 'ten_sec_pause_count']
    ten_sec = pd.merge(ten_sec_pauses,total_rows_per_id, how = 'left', on = 'id')
    ten_sec['ten_sec_pause_to_output_ratio'] = 0  # Initialize ratio column with zeros
    ten_sec.loc[ten_sec['total_actions'] != 0, 'ten_sec_pause_to_output_ratio'] = ten_sec['ten_sec_pause_count'] / ten_sec['total_actions']
    del(ten_sec['total_actions'])
    
    calc_sum_df = pd.merge(punct_summary_df, revision_results, how='left', on = 'id')
    calc_sum_df = pd.merge(calc_sum_df, total_rows_per_id, how = 'left', on = 'id')
    calc_sum_df = pd.merge(calc_sum_df, one_min, how='left', on = 'id')
    calc_sum_df = pd.merge(calc_sum_df, ten_sec, how='left', on = 'id')
    return calc_sum_df

def data_processing(df):
    filtered_df = raw_training_data.dropna()

    #removing data outside of the time threshold plus one minute
    filtered_df = filtered_df.groupby('id').filter(lambda x: (x['up_time'] <= 1860000).all())
    #Creating inter_action_variable
    filtered_df.sort_values(by=['id', 'up_time'], inplace=True)
    filtered_df['inter_action_time'] = filtered_df.groupby('id')['up_time'].diff()
    num_segments = 6

    

    segmented_data_dict = segment_df_creator(filtered_df, num_segments)

    

    for key in segmented_data_dict.keys():
        segmented_data_dict[key] = segmented_data_dict[key].groupby('id', as_index = False).apply(scale_within_group)
        segmented_data_dict[key]['cursor_position'] = segmented_data_dict[key]['cursor_position'].astype(int)
        segmented_data_dict[key]['actions_occured'] = 1
    segmented_data_dict['full'] = filtered_df
    
    essay_dict = {
    "0":None,
    "1":None,
    "2":None,
    "3":None,
    "4":None,
    "5":None,
    "full":None
    }
    for key in essay_dict.keys():

        essay_dict[key] = getEssays(segmented_data_dict[key])
        
    summary_columns = ['total_actions', 'total_words', 'periods_count', 'q_count', 'space_count']

    summary_df = create_sum_stats(filtered_df)
    summary_df_0_5 = create_sum_stats(segmented_data_dict["0"])
    summary_df_0_5.columns = column_list(summary_df_0_5,'0_5')
    summary_df_5_10 = create_sum_stats(segmented_data_dict["1"])
    summary_df_5_10.columns = column_list(summary_df_5_10,'5_10')
    summary_df_10_15 = create_sum_stats(segmented_data_dict["2"])
    summary_df_10_15.columns = column_list(summary_df_10_15,'10_15')
    summary_df_15_20 = create_sum_stats(segmented_data_dict["3"])
    summary_df_15_20.columns = column_list(summary_df_15_20,'15_20')
    summary_df_20_25 = create_sum_stats(segmented_data_dict["4"])
    summary_df_20_25.columns = column_list(summary_df_20_25,'20_25')
    summary_df_25_30 = create_sum_stats(segmented_data_dict["5"])
    summary_df_25_30.columns = column_list(summary_df_25_30,'25_30')

    # List of DataFrames
    summary_df_list = [summary_df, summary_df_0_5, summary_df_5_10,summary_df_10_15, summary_df_15_20,  summary_df_20_25, summary_df_25_30 ]

    # Merge DataFrames on 'key_column'
    full_summary_data_df = reduce(lambda left, right: pd.merge(left, right, on='id', how='outer'), summary_df_list)

    essay_df = essay_output_to_df(essay_dict['full'])
    essay_df_0_5 = essay_output_to_df(essay_dict['0'])
    essay_df_0_5.columns = column_list(essay_df_0_5,'0_5')
    essay_df_5_10 = essay_output_to_df(essay_dict['1'])
    essay_df_5_10.columns = column_list(essay_df_5_10,'5_10')
    essay_df_10_15 = essay_output_to_df(essay_dict['2'])
    essay_df_10_15.columns = column_list(essay_df_10_15,'10_15')
    essay_df_15_20 = essay_output_to_df(essay_dict['3'])
    essay_df_15_20.columns = column_list(essay_df_15_20,'15_20')
    essay_df_20_25 = essay_output_to_df(essay_dict['4'])
    essay_df_20_25.columns = column_list(essay_df_20_25,'20_25')
    essay_df_25_30 = essay_output_to_df(essay_dict['5'])
    essay_df_25_30.columns = column_list(essay_df_25_30,'25_30')

    essay_df_list = [essay_df, essay_df_0_5, essay_df_5_10,essay_df_10_15, essay_df_15_20,  essay_df_20_25, essay_df_25_30 ]

    # Merge DataFrames on 'key_column'
    full_essay_data_df = reduce(lambda left, right: pd.merge(left, right, on='id', how='outer'), essay_df_list)

    calc_stats_df = calc_output_to_df(segmented_data_dict['full'])
    calc_stats_df_0_5 = calc_output_to_df(segmented_data_dict['0'])
    calc_stats_df_0_5.columns = column_list(calc_stats_df_0_5,'0_5')
    calc_stats_df_5_10 = calc_output_to_df(segmented_data_dict['1'])
    calc_stats_df_5_10.columns = column_list(calc_stats_df_5_10,'5_10')
    calc_stats_df_10_15 = calc_output_to_df(segmented_data_dict['2'])
    calc_stats_df_10_15.columns = column_list(calc_stats_df_10_15,'10_15')
    calc_stats_df_15_20 = calc_output_to_df(segmented_data_dict['3'])
    calc_stats_df_15_20.columns = column_list(calc_stats_df_15_20,'15_20')
    calc_stats_df_20_25 = calc_output_to_df(segmented_data_dict['4'])
    calc_stats_df_20_25.columns = column_list(calc_stats_df_20_25,'20_25')
    calc_stats_df_25_30 = calc_output_to_df(segmented_data_dict['5'])
    calc_stats_df_25_30.columns = column_list(calc_stats_df_25_30,'25_30')


    calc_df_list = [calc_stats_df, calc_stats_df_0_5, calc_stats_df_5_10,calc_stats_df_10_15, calc_stats_df_15_20,  calc_stats_df_20_25, calc_stats_df_25_30 ]

    # Merge DataFrames on 'key_column'
    full_calc_data_df = reduce(lambda left, right: pd.merge(left, right, on='id', how='outer'), calc_df_list)

    
    all_df_list = [full_calc_data_df, full_essay_data_df, full_summary_data_df, raw_training_scores ]

    # Merge DataFrames on 'key_column'
    training_dataset_feature_df = reduce(lambda left, right: pd.merge(left, right, on='id', how='outer'), all_df_list)


Some Initial Exploration

In [ ]:

nan_counts = raw_training_data.isna().sum()
print(nan_counts)
nan_rows = raw_training_data[raw_training_data.isnull().any(axis=1)]
nan_rows


raw_training_data.columns #columns included in the data

len(raw_training_data)#number of data points

raw_training_data.head(15)
len((raw_training_data['id']).unique()) #number of unique essays

#information on the number of words per essay
# Group the DataFrame by 'id' and get the last row of each group
last_rows = raw_training_data.groupby('id').last()

# Extract the 'word count' column from the last rows
word_counts = last_rows['word_count']
print(max(word_counts))
print(min(word_counts))

print(word_counts.mean())
print(word_counts.std())


#information on the numer of actions per essay
value_counts = raw_training_data['id'].value_counts()

# Calculate the average number of occurrences
average_count = value_counts.mean()
sd_counts = value_counts.std()
print(average_count)
print(sd_counts)
print(max(value_counts))
print(min(value_counts))

raw_training_data['activity'].value_counts()

minvals = raw_training_data.groupby('id').min()
maxvals = raw_training_data.groupby('id').max()

min(maxvals['up_time'])

print(len(raw_training_data['id'].unique()))
filtered_df = raw_training_data.groupby('id').filter(lambda x: (x['up_time'] <= 1860000).all())
print(len(filtered_df['id'].unique()))




Data Filtering

In [ ]:
#removing_na_columns
filtered_df = raw_training_data.dropna()

#removing data outside of the time threshold plus one minute
filtered_df = filtered_df.groupby('id').filter(lambda x: (x['up_time'] <= 1860000).all())
#Creating inter_action_variable
filtered_df.sort_values(by=['id', 'up_time'], inplace=True)
filtered_df['inter_action_time'] = filtered_df.groupby('id')['up_time'].diff()


#setting all events without at least 10000 instances to "other"


Creation of Essay Sections

In [ ]:
%%time
num_segments = 6

def segment_df_creator(df, num_segments):

    segment_dict = {}
    last_action_time = 1860000
    interval_len = int(round(last_action_time/num_segments, 0))
    
    
    for i in range(num_segments):
        if i == 0: 
            segment_dict[str(i)] = df[df['up_time'] < interval_len].reset_index()
        
            
        elif i == max(range(num_segments)):
            segment_dict[str(i)] = df[df['up_time'] >interval_len*max(range(num_segments))].reset_index()
        
        else: 
            segment_dict[str(i)] = df[(df['up_time']>interval_len*(i-1)) & (df['up_time']<interval_len*(i)) ] .reset_index()
        
    return segment_dict

segmented_data_dict = segment_df_creator(filtered_df, num_segments)

def scale_within_group(group):
    min_value = group['cursor_position'].min()
    max_value = group['cursor_position'].max()
    group['cursor_position'] = group['cursor_position'] - min_value
    return group

for key in segmented_data_dict.keys():
    segmented_data_dict[key] = segmented_data_dict[key].groupby('id', as_index = False).apply(scale_within_group)
    segmented_data_dict[key]['cursor_position'] = segmented_data_dict[key]['cursor_position'].astype(int)
    segmented_data_dict[key]['actions_occured'] = 1
segmented_data_dict['full'] = filtered_df


In [ ]:
len(raw_training_data)

In [ ]:
print(len(segmented_data_dict['0']['id'].unique()))
print(len(segmented_data_dict['1']['id'].unique()))
print(len(segmented_data_dict['2']['id'].unique()))
print(len(segmented_data_dict['3']['id'].unique()))
print(len(segmented_data_dict['4']['id'].unique()))
print(len(segmented_data_dict['5']['id'].unique()))


In [ ]:
display(segmented_data_dict['full'][segmented_data_dict['full']['id'] == '0144e4d5'])

Addition of Function to Write Out Essays for Easy Analysis

In [ ]:
#####CODE FROM: https://www.kaggle.com/code/kawaiicoderuwu/essay-contructor

def getEssays(df):
    # Copy required columns
    textInputDf = df[['id', 'activity', 'cursor_position', 'text_change']].copy()
    
    # Get rid of text inputs that make no change
    # Note: Shift was unpreditcable so ignored
    textInputDf = textInputDf[textInputDf.activity != 'Nonproduction']

    # Get how much each Id there is
    valCountsArr = textInputDf['id'].value_counts(sort=False).values

    # Holds the final index of the previous Id
    lastIndex = 0

    # Holds all the essays
    essaySeries = pd.Series()

    # Fills essay series with essays
    for index, valCount in enumerate(valCountsArr):

        # Indexes down_time at current Id
        currTextInput = textInputDf[['activity', 'cursor_position', 'text_change']].iloc[lastIndex : lastIndex + valCount]

        # Update the last index
        lastIndex += valCount

        # Where the essay content will be stored
        essayText = ""

        
        # Produces the essay
        for Input in currTextInput.values:
            
            # Input[0] = activity
            # Input[2] = cursor_position
            # Input[3] = text_change
            
            # If activity = Replace
            if Input[0] == 'Replace':
                # splits text_change at ' => '
                replaceTxt = Input[2].split(' => ')
                
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                continue

                
            # If activity = Paste    
            if Input[0] == 'Paste':
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                continue

                
            # If activity = Remove/Cut
            if Input[0] == 'Remove/Cut':
                # DONT TOUCH
                essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                continue

                
            # If activity = Move...
            if "M" in Input[0]:
                # Gets rid of the "Move from to" text
                croppedTxt = Input[0][10:]
                
                # Splits cropped text by ' To '
                splitTxt = croppedTxt.split(' To ')
                
                # Splits split text again by ', ' for each item
                valueArr = [item.split(', ') for item in splitTxt]
                
                # Move from [2, 4] To [5, 7] = (2, 4, 5, 7)
                moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))

                # Skip if someone manages to activiate this by moving to same place
                if moveData[0] != moveData[2]:
                    # Check if they move text forward in essay (they are different)
                    if moveData[0] < moveData[2]:
                        # DONT TOUCH
                        essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                    else:
                        # DONT TOUCH
                        essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                continue
                
                
            # If just input
            # DONT TOUCH
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]

            
        # Sets essay at index  
        essaySeries[index] = essayText
     
    
    # Sets essay series index to the ids
    essaySeries.index =  textInputDf['id'].unique()
    
    
    # Returns the essay series
    return essaySeries

Writing out of essays - by section and total

In [ ]:

#THE OUTPUT IS LOADED IN NEXT CELL

essay_dict = {
    "0":None,
    "1":None,
    "2":None,
    "3":None,
    "4":None,
    "5":None,
    "full":None
}
for key in essay_dict.keys():
    
    essay_dict[key] = getEssays(segmented_data_dict[key])


Essay-Formatted Data Load

In [ ]:
#Saving the data
"""
with open('essays.pickle', 'wb') as handle:
    pickle.dump(essay_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
"""

Loading the Essay formatted Data

In [ ]:
with open('/kaggle/input/essay-segments-written-form2/essays.pickle', 'rb') as handle:
    essay_dict = pickle.load(handle)

In [ ]:
display(essay_dict['1'])

In [ ]:
print(len(essay_dict['0']['id'].unique()))
print(len(essay_dict['1']['id'].unique()))
print(len(essay_dict['2']['id'].unique()))
print(len(essay_dict['3']['id'].unique()))
print(len(essay_dict['4']['id'].unique()))
print(len(essay_dict['5']['id'].unique()))


SUMMARY INFORMATION

In [ ]:
summary_columns = ['total_actions', 'total_words', 'periods_count', 'q_count', 'space_count']

     
        

    
summary_df = create_sum_stats(filtered_df)
summary_df_0_5 = create_sum_stats(segmented_data_dict["0"])
summary_df_0_5.columns = column_list(summary_df_0_5,'0_5')
summary_df_5_10 = create_sum_stats(segmented_data_dict["1"])
summary_df_5_10.columns = column_list(summary_df_5_10,'5_10')
summary_df_10_15 = create_sum_stats(segmented_data_dict["2"])
summary_df_10_15.columns = column_list(summary_df_10_15,'10_15')
summary_df_15_20 = create_sum_stats(segmented_data_dict["3"])
summary_df_15_20.columns = column_list(summary_df_15_20,'15_20')
summary_df_20_25 = create_sum_stats(segmented_data_dict["4"])
summary_df_20_25.columns = column_list(summary_df_20_25,'20_25')
summary_df_25_30 = create_sum_stats(segmented_data_dict["5"])
summary_df_25_30.columns = column_list(summary_df_25_30,'25_30')

# List of DataFrames
summary_df_list = [summary_df, summary_df_0_5, summary_df_5_10,summary_df_10_15, summary_df_15_20,  summary_df_20_25, summary_df_25_30 ]

# Merge DataFrames on 'key_column'
full_summary_data_df = reduce(lambda left, right: pd.merge(left, right, on='id', how='outer'), summary_df_list)



Creation of Functions to Calculate Desired Features Based on Literature

In [ ]:

essay_df = essay_output_to_df(essay_dict['full'])
essay_df_0_5 = essay_output_to_df(essay_dict['0'])
essay_df_0_5.columns = column_list(essay_df_0_5,'0_5')
essay_df_5_10 = essay_output_to_df(essay_dict['1'])
essay_df_5_10.columns = column_list(essay_df_5_10,'5_10')
essay_df_10_15 = essay_output_to_df(essay_dict['2'])
essay_df_10_15.columns = column_list(essay_df_10_15,'10_15')
essay_df_15_20 = essay_output_to_df(essay_dict['3'])
essay_df_15_20.columns = column_list(essay_df_15_20,'15_20')
essay_df_20_25 = essay_output_to_df(essay_dict['4'])
essay_df_20_25.columns = column_list(essay_df_20_25,'20_25')
essay_df_25_30 = essay_output_to_df(essay_dict['5'])
essay_df_25_30.columns = column_list(essay_df_25_30,'25_30')

essay_df_list = [essay_df, essay_df_0_5, essay_df_5_10,essay_df_10_15, essay_df_15_20,  essay_df_20_25, essay_df_25_30 ]

# Merge DataFrames on 'key_column'
full_essay_data_df = reduce(lambda left, right: pd.merge(left, right, on='id', how='outer'), essay_df_list)



In [ ]:


calc_stats_df = calc_output_to_df(segmented_data_dict['full'])
calc_stats_df_0_5 = calc_output_to_df(segmented_data_dict['0'])
calc_stats_df_0_5.columns = column_list(calc_stats_df_0_5,'0_5')
calc_stats_df_5_10 = calc_output_to_df(segmented_data_dict['1'])
calc_stats_df_5_10.columns = column_list(calc_stats_df_5_10,'5_10')
calc_stats_df_10_15 = calc_output_to_df(segmented_data_dict['2'])
calc_stats_df_10_15.columns = column_list(calc_stats_df_10_15,'10_15')
calc_stats_df_15_20 = calc_output_to_df(segmented_data_dict['3'])
calc_stats_df_15_20.columns = column_list(calc_stats_df_15_20,'15_20')
calc_stats_df_20_25 = calc_output_to_df(segmented_data_dict['4'])
calc_stats_df_20_25.columns = column_list(calc_stats_df_20_25,'20_25')
calc_stats_df_25_30 = calc_output_to_df(segmented_data_dict['5'])
calc_stats_df_25_30.columns = column_list(calc_stats_df_25_30,'25_30')


calc_df_list = [calc_stats_df, calc_stats_df_0_5, calc_stats_df_5_10,calc_stats_df_10_15, calc_stats_df_15_20,  calc_stats_df_20_25, calc_stats_df_25_30 ]

# Merge DataFrames on 'key_column'
full_calc_data_df = reduce(lambda left, right: pd.merge(left, right, on='id', how='outer'), calc_df_list)


In [ ]:


pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
df = segmented_data_dict['full'][segmented_data_dict['full']['id'] == '0178a105']
total_rows_per_id = df.groupby('id').size().reset_index()
total_rows_per_id.columns = ['id', 'total_actions']
display(full_calc_data_df[full_calc_data_df['id'] == '0178a105'])


In [ ]:
all_df_list = [full_calc_data_df, full_essay_data_df, full_summary_data_df, raw_training_scores ]

# Merge DataFrames on 'key_column'
training_dataset_feature_df = reduce(lambda left, right: pd.merge(left, right, on='id', how='outer'), all_df_list)


In [ ]:
for col in training_dataset_feature_df.columns:
    
    if 'total_actions' in col:
        training_dataset_feature_df[col] = training_dataset_feature_df[col].fillna(0)
    elif 'min_pause' in col: 
        training_dataset_feature_df[col] = training_dataset_feature_df[col].fillna(0)
    elif 'sec_pause' in col: 
        training_dataset_feature_df[col] = training_dataset_feature_df[col].fillna(0)
    else: pass

In [ ]:


training_data['action_occured_0_5'] = np.where(training_data['id'].isin(segmented_data_dict['0']['id'].unique()), 1, 0)
    
training_data['action_occured_5_10'] = np.where(training_data['id'].isin(segmented_data_dict['1']['id'].unique()), 1, 0)

training_data['action_occured_10_15'] = np.where(training_data['id'].isin(segmented_data_dict['2']['id'].unique()), 1, 0)

training_data['action_occured_15_20'] = np.where(training_data['id'].isin(segmented_data_dict['3']['id'].unique()), 1, 0)

training_data['action_occured_20_25'] = np.where(training_data['id'].isin(segmented_data_dict['4']['id'].unique()), 1, 0)

training_data['action_occured_25_30'] = np.where(training_data['id'].isin(segmented_data_dict['5']['id'].unique()), 1, 0)


In [ ]:
training_data['action_occured_0_5'].value_counts()
training_data['action_occured_20_25'].value_counts()



In [ ]:
from sklearn.impute import KNNImputer
import numpy as np
data = training_data.copy()
del(data['id'])

# Specify the index of the column to be ignored during imputation
  # Change this to the column name you want to ignore

# Copy the DataFrame for imputation
data_copy = data.copy()

# Convert the column to ignore to NumPy for imputation
column_index =0
data_array = data_copy.values


# Perform imputation
imputer = KNNImputer(n_neighbors=2, weights='distance', metric='nan_euclidean')
imputed_data_array = imputer.fit_transform(data_array)



# Convert back to Pandas DataFrame
columns = list(data_copy.columns)

final_df = pd.DataFrame(imputed_data_array, columns=columns)



In [ ]:
final_df['id'] = training_data['id']

In [ ]:
len(final_df.dropna())

In [ ]:
final_df.to_csv('imputed_training_data.csv')